In [ ]:
#

```
function ClickConnect(){
    console.log("Working!"); 
    document.querySelector("colab-run-button").click()
}
setInterval(ClickConnect, 60000)
```

In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-1ltDoEYlNzwRF84HMIcKbDucgmId3cP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-1ltDoEYlNzwRF84HMIcKbDucgmId3cP" -O /content/dataset.zip && rm -rf /tmp/cookies.txt
!unzip -q /content/dataset.zip -d /content

--2021-04-19 14:08:59--  https://docs.google.com/uc?export=download&confirm=SFvq&id=1-1ltDoEYlNzwRF84HMIcKbDucgmId3cP
Resolving docs.google.com (docs.google.com)... 74.125.195.100, 74.125.195.138, 74.125.195.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-8s-docs.googleusercontent.com/docs/securesc/8cl75e20nppnbi5ifubpkm0o17j8fbe7/vc9usjl8plm32lilskp9bbqa2qsoaq0h/1618841325000/03816308716299652124/08908538557672240542Z/1-1ltDoEYlNzwRF84HMIcKbDucgmId3cP?e=download [following]
--2021-04-19 14:08:59--  https://doc-00-8s-docs.googleusercontent.com/docs/securesc/8cl75e20nppnbi5ifubpkm0o17j8fbe7/vc9usjl8plm32lilskp9bbqa2qsoaq0h/1618841325000/03816308716299652124/08908538557672240542Z/1-1ltDoEYlNzwRF84HMIcKbDucgmId3cP?e=download
Resolving doc-00-8s-docs.googleusercontent.com (doc-00-8s-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting

In [2]:
!pip install pytorch_lightning neptune-client > /dev/null

In [3]:
import numpy as np
from PIL import Image
from pathlib import Path
import urllib.request
import zipfile
import random
import gc

import torch
import torch.utils.data as data
from torchvision import transforms
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers.neptune import NeptuneLogger
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class InterjectionDataset(data.Dataset):
  def __init__(self, dir_path, input_size, meta_df):
    super().__init__()
    self.dir_path = dir_path
    self.input_size = input_size
    self.meta_df = meta_df
    self.len = len(self.meta_df)
  
  def __len__(self):
    return self.len
  
  def __getitem__(self, index):
    row = self.meta_df.iloc[index]
    image_name = row.image
    p = os.path.join(self.dir_path, image_name)
    image = Image.open(p)
    image = image.resize(self.input_size)
    image = np.array(image)
    image = image.reshape(512, 512, 1)
    image = np.transpose(image, (2, 0, 1))
    image = torch.from_numpy(image)
    image = image.type(torch.FloatTensor)

    start = row.start
    end = row.end
    label = np.zeros(512)
    label[start:end] = 1
    label = torch.from_numpy(label)
    label = label.type(torch.FloatTensor)

    return image, label

In [5]:
dir_path = "/content/content/dataset_image"
image_path = os.path.join(dir_path,"image")
meta_path = os.path.join(dir_path,"interjection_data.csv")

with open(meta_path, 'r', encoding='UTF-8') as csv:
  meta_df = pd.read_csv(csv)
  dataset = InterjectionDataset(image_path, (512,512), meta_df)

test_size = int(len(dataset) * 0.2)
valid_size = test_size
train_size = len(dataset) - (test_size + valid_size)
splited_dataset = data.random_split(dataset, [train_size, test_size, valid_size], generator=torch.Generator().manual_seed(0))
train_dataset = splited_dataset[0]
test_dataset = splited_dataset[1]
valid_dataset = splited_dataset[2]

In [6]:
def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
  """1x1 convolution"""
  return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
  """3x3 convolution with padding"""
  return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                    padding=dilation, groups=groups, bias=False, dilation=dilation)

class BasicBlock(nn.Module):
  expansion: int = 1

  def __init__(
    self,
    inplanes: int,
    planes: int,
    stride: int = 1,
    downsample: Optional[nn.Module] = None,
    groups: int = 1,
    base_width: int = 64,
    dilation: int = 1,
    norm_layer: Optional[Callable[..., nn.Module]] = None
  ) -> None:
    super(BasicBlock, self).__init__()
    if norm_layer is None:
        norm_layer = nn.BatchNorm2d
    if groups != 1 or base_width != 64:
        raise ValueError('BasicBlock only supports groups=1 and base_width=64')
    if dilation > 1:
        raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
    # Both self.conv1 and self.downsample layers downsample the input when stride != 1
    self.conv1 = conv3x3(inplanes, planes, stride)
    self.bn1 = norm_layer(planes)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = conv3x3(planes, planes)
    self.bn2 = norm_layer(planes)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x: Tensor) -> Tensor:
    identity = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)

    if self.downsample is not None:
        identity = self.downsample(x)

    out += identity
    out = self.relu(out)

    return out

In [105]:
class InterjectionModel(pl.LightningModule):
  def __init__(self, frequency_dim , hidden_dim, batch_size):
    super(InterjectionModel, self).__init__()
    self.frequency_dim = frequency_dim
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size    
    self.num_layers = 2
    self.lstm = nn.LSTM(input_size=frequency_dim, hidden_size=hidden_dim, num_layers=self.num_layers, batch_first=True, bidirectional=True)
    self.layer1 = BasicBlock(1,64)
    self.layer2 = BasicBlock(64,64)
    # self.layer3 = conv1x1(16,32)
    # self.layer4 = BasicBlock(32,32)
    self.layer5 = conv1x1(64, 1)

    self.softmax = nn.LogSoftmax()
    self.criterion = nn.BCEWithLogitsLoss()

  def forward(self, images):
    # images.size() = (batch_size × 1 × frequency_dim × hidden_dim)

    x = self.layer1(images)
    x = self.layer2(x)
    x = self.layer2(x)
    x = self.layer2(x)
    x = self.layer2(x)
    x = self.layer2(x)
    x = self.layer2(x)
    x = self.layer2(x)

    # x = self.layer3(x)
    # x = self.layer4(x)
    # x = self.layer4(x)
    # x = self.layer4(x)
    x = self.layer5(x)

    x = images
    # x.size() = (batch_size × hidden_dim × hidden_dim)
    x = x.view(self.batch_size, self.hidden_dim, self.hidden_dim)

    _, lstm_out = self.lstm(x)
    # lstm_out[0].size() = ((num_layers × direction)× batch_size × hidden_dim)
    last_layer = (self.num_layers - 1) * 2
    lstm_out_mean = (lstm_out[0][last_layer] + lstm_out[0][last_layer + 1]) / 2
    result = lstm_out_mean.squeeze()
    # result.size() = (batch_size × hidden_dim)
    return result

  def training_step(self, batch, batch_nb):
    x, y = batch
    y_hat = self(x)
    loss = self.criterion(y_hat, y)
    self.log('train_loss', loss)
    return {'loss': loss}
  
  def validation_step(self, batch, batch_nb):
    x, y = batch
    y_hat = self(x)
    return {'val_loss': self.criterion(y_hat, y)}

  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    self.log('val_loss', avg_loss)
    gc.collect()

  def test_step(self, batch, batch_nb):
      x, y = batch
      y_hat = self(x)
      return {'test_loss': self.criterion(y_hat, y)}

  def test_epoch_end(self, outputs):
    avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    self.log('val_loss', avg_loss, prog_bar=True,)

  def configure_optimizers(self):
    # REQUIRED
    # can return multiple optimizers and learning_rate schedulers
    # (LBFGS it is automatically supported, no need for closure function)
    return optim.Adam(self.parameters(), lr=0.02)

  def train_dataloader(self):
    return data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2, drop_last=True)

  def val_dataloader(self):
    return data.DataLoader(valid_dataset, batch_size=self.batch_size, num_workers=2, drop_last=True)

  def test_dataloader(self):
    return data.DataLoader(test_dataset, batch_size=self.batch_size, num_workers=2, drop_last=True)

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [116]:
from getpass import getpass

api_key = getpass('Enter your private Neptune API token: ')

Enter your private Neptune API token: ··········


In [ ]:
# Project
PROJECT_NAME = 'hourglasshoro/test'
EXPERIMENT_NAME = ''
TAGS = []

# Param
BATCH_SIZE = 4

LightningModule_Params = {'image_size': 512,
                          'n_lstm_layer': 2,
                          'lstm_bidirection': True,
                          'learning_rate': 0.02,
                          'res_block_type': 'Basic',
                          'res_block_layer_1': 64,
                          'res_block_layer_1_num': 8}

LightningDataModule_Params = {'batch_size': BATCH_SIZE,
                              'num_workers': 2}

LearningRateMonitor_Params = {'logging_interval': 'epoch'}

ModelCheckpoint_Params = {'filename': '{epoch:02d}-{val_loss:.2f}',
                          'save_weights_only': True,
                          'monitor': 'val_loss',
                          'period': 1}

EarlyStopping_Params = {'monitor': 'val_loss'}

Trainer_Params = {'max_epochs': 30}

ALL_PARAMS = {**LightningModule_Params,
              **LightningDataModule_Params,
              **LearningRateMonitor_Params,
              **EarlyStopping_Params,
              **ModelCheckpoint_Params,
              **Trainer_Params}

neptune_logger = NeptuneLogger(
    api_key=api_key,
    project_name=PROJECT_NAME,
    close_after_fit=False,
    experiment_name=EXPERIMENT_NAME,
    params=ALL_PARAMS,
    tags=TAGS,
    )

model = InterjectionModel(512, 512, BATCH_SIZE)
early_stop = EarlyStopping(monitor='val_loss')
checkpoint = ModelCheckpoint(monitor='val_loss', filename='{epoch:02d}-{val_loss:.2f}', save_weights_only=True)
root_dir = '/content/drive/MyDrive/' + PROJECT_NAME + '/' + EXPERIMENT_NAME
trainer = pl.Trainer(max_epochs=30, gpus=1, default_root_dir=root_dir, callbacks=[early_stop, checkpoint], logger=neptune_logger)
gc.collect()  
trainer.fit(model)   

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://ui.neptune.ai/hourglasshoro/test/e/TES-10



  | Name      | Type              | Params
------------------------------------------------
0 | lstm      | LSTM              | 10.5 M
1 | layer1    | BasicBlock        | 37.7 K
2 | layer2    | BasicBlock        | 74.0 K
3 | layer5    | Conv2d            | 64    
4 | softmax   | LogSoftmax        | 0     
5 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.456    Total estimated model params size (MB)


In [ ]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 0.54933100938797}
--------------------------------------------------------------------------------


[{'val_loss': 0.54933100938797}]

In [ ]:
neptune_logger.experiment.stop()

## Not use Pytorch Lightning

In [ ]:
class LSTMClassifier(nn.Module):
  def __init__(self, frequency_dim , hidden_dim, batch_size):
    super(LSTMClassifier, self).__init__()
    self.frequency_dim = frequency_dim
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size
    self.lstm = nn.LSTM(input_size=frequency_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True)
    # self.lstm = nn.LSTM(input_size=frequency_dim, hidden_size=hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
    self.softmax = nn.LogSoftmax()
  
  def forward(self, images):
    # images.size() = (batch_size × 1 × frequency_dim × hidden_dim)
    images = images.view(self.batch_size, self.frequency_dim, self.hidden_dim)
    # images.size() = (batch_size × frequency_dim × hidden_dim)
    _, lstm_out = self.lstm(images)
    # lstm_out[0].size() = (1 × batch_size × hidden_dim)
    # result = self.softmax(lstm_out[0].squeeze())
    # result.size() = (batch_size × hidden_dim)

    result = lstm_out[0].squeeze()

    return result

In [ ]:
model = LSTMClassifier(512, 512, BATCH_SIZE).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(50):
  all_loss = 0
  train_dataloader = data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, drop_last=True
  )
  for i , (images, labels) in enumerate(train_dataloader):

    # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
    image_tensor = torch.tensor(images, device=device)
    # category_tensor.size() = (batch_size × 1)なので、squeeze()
    label_tensor = torch.tensor(labels, device=device).squeeze()

    out = model(image_tensor)
    batch_loss = criterion(out, label_tensor)

    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    all_loss += batch_loss.item()
  print(epoch+1, all_loss)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


1 111.41851443052292
2 111.2116351723671
3 111.16537064313889
4 111.16139322519302
5 111.16120076179504
6 111.16709697246552
7 111.16564673185349
8 111.17030203342438
9 111.17257761955261
10 111.171446621418
11 111.17331147193909
12 111.17211610078812
13 111.17215651273727
14 111.17157238721848
15 111.17320388555527
16 111.1725537776947
17 111.17196184396744
18 111.17305034399033
19 111.17205286026001
20 111.1707011461258
21 111.14725542068481
22 111.14445626735687
23 111.1438837647438
24 111.14277589321136
25 111.14232015609741
26 111.14104413986206
27 111.14273911714554
28 111.14228075742722
29 111.14271742105484
30 111.14269280433655
31 111.14145314693451
32 111.14332270622253
33 111.1345984339714
34 111.11222237348557
35 111.1142349243164
36 111.11433750391006
37 111.11308234930038
38 111.11320859193802
39 111.11185383796692
40 111.11280363798141
41 111.11339288949966
42 111.11564862728119
43 111.11343276500702
44 111.11402040719986
45 111.11277449131012
46 111.11353552341461
47 11